# Fintech Model for company stocks and where to invest

In [1]:
import pandas as pd
import numpy as np

In [2]:
amazon = pd.read_csv("Amazon Stock One Year Historical Data.csv")
apple = pd.read_csv("Apple 1 Year Historical Data.csv")
microsoft = pd.read_csv("Microsoft stocks 1 year data.csv")

In [3]:
amazon['Company'] = 'Amazon'
apple['Company'] = 'Apple'
microsoft['Company'] = 'Microsoft'

In [4]:
for df in [amazon, apple, microsoft]:
    df.columns = df.columns.str.strip().str.lower()
    df['date'] = pd.to_datetime(df['date'])
    df.sort_values('date', inplace=True)

### Combine all Datasets 

In [5]:
combined = pd.concat([amazon, apple, microsoft], ignore_index=True)

## Daily Return

In [6]:
combined['daily_return'] = combined.groupby('company')['close'].pct_change()

## 20 - 50 Days moving Average

In [7]:
combined['ma_20'] = combined.groupby('company')['close'].transform(lambda x: x.rolling(window=20).mean())
combined['ma_50'] = combined.groupby('company')['close'].transform(lambda x: x.rolling(window=50).mean())

## Rolling Volatality

In [8]:
combined['volatility'] = combined.groupby('company')['daily_return'].transform(lambda x: x.rolling(window=20).std())

## Normalize closing price

In [9]:
combined['normalized_close'] = combined.groupby('company')['close'].transform(lambda x: x / x.iloc[0])

### Save dataset

In [10]:
combined.to_csv("Cleaned_Stock_Data.csv", index=False)

# Investment Simulation model

This is where the CAGR, Adjusted CAGR and Projected Value is being calculated

In [11]:
def simulate_investment(df, investment_amount, duration_days, risk_level):
    risk_adjustment = {'Low': 0.9, 'Medium': 1.0, 'High': 1.1}
    results = []

    for company in df['company'].unique():
        company_df = df[df['company'] == company].copy()
        company_df = company_df.sort_values('date')

        start_price = company_df['close'].iloc[0]
        end_price = company_df['close'].iloc[min(duration_days, len(company_df)-1)]

        cagr = ((end_price / start_price) ** (365 / duration_days)) - 1
        adjusted_cagr = cagr * risk_adjustment[risk_level]
        projected_value = investment_amount * ((1 + adjusted_cagr) ** (duration_days / 365))

        results.append({
            'Company': company,
            'Start Price': start_price,
            'End Price': end_price,
            'CAGR': round(cagr, 4),
            'Adjusted CAGR': round(adjusted_cagr, 4),
            'Projected Value': round(projected_value, 2)
        })

    return pd.DataFrame(results)

#### Small example to show working

In [12]:
simulation = simulate_investment(combined, investment_amount=100000, duration_days=180, risk_level='Medium')
simulation.to_csv("Investment_Simulation.csv", index=False)

In [13]:
simulation

,Company,Start Price,End Price,CAGR,Adjusted CAGR,Projected Value
0,Amazon,161.02,186.54,0.3476,0.3476,115848.96
1,Apple,209.27,208.37,-0.0087,-0.0087,99569.93
2,Microsoft,395.15,387.30,-0.0399,-0.0399,98013.41
